# Libraries

In [6]:
from search import ( # Bases para construcción de problemas
    Problem, Node, Graph, UndirectedGraph,
    SimpleProblemSolvingAgentProgram,
    GraphProblem
)

from search import ( # Algoritmos de búsqueda no informada
    tree_search, graph_search, best_first_graph_search,
    breadth_first_tree_search, breadth_first_search,
    depth_first_tree_search, depth_first_graph_search,
    depth_limited_search, iterative_deepening_search,
    uniform_cost_search,
    compare_searchers,astar_search
)

# Some Helping Methods

In [2]:
import time

class Tick():
    def __init__(self):
        self.t = time.time()

    def tick(self):
        self.t = time.time()
        return abs(self.t)

    def tock(self):
        return abs(self.t - time.time())

def deep_tuple(l):
    return tuple([tuple(x) for x in l])

def tuple_to_list(tuple):
    n = len(tuple)
    l = [0] * n
    for i in range(n):
        l[i] = [0]*n
        for j in range(n):
            l[i][j] =tuple[i][j]
    return l

def print_state(state):
    n = len(state)
    print(" ",end=' ')
    for i in range (0,n):
        print(i,end=' ')
    print(' ')
    for y in range(n):
        print(y, end=' ')
        for x in range(n):
            print(state[x][y], end=' ')
        print()

def print_result(name,node):
    print("----------\n",name,":\n")
    for node in node.path():
        print()
        print("Accion: ",node.action,"\n")
        print_state(node.state)

def num_neighbors(matrix, x, y, r):
    a=matrix
    c=0
    m = len(a)

    if (x-r >= 0):
        xmin = x-r
    else :
        xmin = 0
    if (x+r < m):
        xmax = x+r+1
    else:
        xmax = m

    if (y-r >= 0):
        ymin = y-r
    else :
        ymin = 0
    if (y+r < m):
        ymax = y+r+1
    else:
        ymax = m

    for i in range(ymin, ymax):
        for j in range(xmin, xmax):
            if (matrix[i][j] == 1):
                c+=1
            else:
                pass
    return c


# Some Problems

In [7]:
def easy_matrix():
    a= [[0,0,1,0],[0,1,0,1],[0,0,1,0], [0,0,0,0]]
    return a

def hard_matrix():
    a= [[1,1,1,1],[1,1,1,1],[1,1,1,1], [1,1,1,1]]
    return a

def easiest_matrix(n=4):
    return empty_matrix(n)

def empty_matrix(n=4):
    a = [0] * n
    for i in range(n):
        a[i] = [0] * n
    return a

# The Problem Class

In [8]:
class CleanPuzzle(Problem):

    def __init__(self, inicial=easiest_matrix(4), h=None):
        n = len(inicial)
        Problem.__init__(self, deep_tuple(inicial), deep_tuple(empty_matrix(n)))
        self.n = n
        self.estado = deep_tuple(inicial)
        self.h = h

    def actions(self, estado):
        accs = []
        for i in range(self.n):
            for j in range(self.n):
                accs.append((i,j))
        return accs

    def click(self,x,y,estado):
        next_estado = tuple_to_list(estado)
        next_estado = self.toggle(x,y-1,next_estado) # up
        next_estado = self.toggle(x-1,y,next_estado) # left
        next_estado = self.toggle(x,y+1,next_estado) # down
        next_estado = self.toggle(x+1,y,next_estado) # right
        return deep_tuple(next_estado)

    def toggle(self,x,y,estado):
        next_estado = estado
        if(x == -1 or y == -1 or x == self.n or y == self.n):
            return next_estado
        else:
            next_estado[x][y] = int(not(estado[x][y]))
            return next_estado

    def result(self, estado, accion):
        return self.click(accion[0],accion[1],estado)

# Herustic 1

In [9]:
def h1(node):
    "Diferencia entre meta y estado actual"
    current = node.state
    toSolve = 0
    for x,coorX in enumerate(current):
        for y,coorY in enumerate(coorX):
            toSolve = toSolve + current[x][y]
    return toSolve

# Heuristic 2

In [10]:
def h2(node):
    "Diferencia entre meta y estado actual"
    n = len(node.state)
    action = node.action
    if(action is None):
        return 100
    x = action[0]
    y = action[1]
    current = node.state
    up = 0
    if (y > 0):
        up = current[x][y-1]
    left =  0
    if (x > 0):
        left = current[x-1][y]
    down = 0
    if (y <n-1):
        down = current[x][y+1]
    right = 0
    if (x<n-1):
        right = current[x+1][y]
    count = up+left+down+right

    if(count == 0):
        return 0
    if(count == 1 or count == 3 or count == 4):
        return 100
    if(count == 2):
        eucO = abs(0-x) + abs(0-y)
        euc1 = abs(0-x) + abs(n-y)
        euc2 = abs(n-x) + abs(n-y)
        euc3 = abs(n-x) + abs(n-y)
        return min(eucO,euc1,euc2,euc3)

    return 100

# Heuristic 3

In [11]:
def h3(node):
    if(node.action is None):
        return 100
    else:
        return num_neighbors(node.state, node.action[0], node.action[1], 2)

In [8]:
problem = CleanPuzzle(hard_matrix(),h3)

n = astar_search(problem,problem.h)
print("\nHeuristica 3\n",n.solution())


Heuristica 3
 [(0, 0), (0, 1), (3, 0), (2, 3), (1, 3), (3, 1)]


In [6]:
problem = CleanPuzzle(hard_matrix())

n = breadth_first_search(problem)
print("\nBlind\n",n.solution())


Blind
 [(0, 0), (0, 1), (1, 3), (2, 3), (3, 0), (3, 1)]


In [10]:
problem = CleanPuzzle(hard_matrix(),h1)

n = astar_search(problem,problem.h)
print("\nHeuristica1\n",n.solution())


Heuristica1
 [(1, 1), (1, 2), (1, 0), (1, 3), (3, 2), (2, 1), (3, 0), (3, 1), (2, 2), (3, 3)]


In [11]:
problem = CleanPuzzle(hard_matrix(),h2)

n = astar_search(problem,problem.h)
print("\nHeuristica2\n",n.solution())


Heuristica2
 [(1, 0), (0, 2), (0, 3), (2, 0), (3, 2), (3, 3)]


In [12]:
problem = CleanPuzzle(hard_matrix(),h3)

n = astar_search(problem,problem.h)
print("\nHeuristica3\n",n.solution())


Heuristica3
 [(0, 0), (0, 1), (3, 0), (2, 3), (1, 3), (3, 1)]


In [25]:
ones_matrix = [[1,1,1,1],[1,1,1,1],[1,1,1,1], [1,1,1,1]]

t = Tick()

t.tick()
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix)
n = breadth_first_search(problem)
print("\nBlind\n",n.solution())
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix,h1)
n = astar_search(problem,problem.h)
print("\nHeuristica 1\n",n.solution())
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix,h2)
n = astar_search(problem,problem.h)
print("\nHeuristica 2\n",n.solution())
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix,h3)
n = astar_search(problem,problem.h)
print("\nHeuristica 3\n",n.solution())
print(t.tock())
t.tick()

2.1457672119140625e-05



Blind
 [(0, 0), (0, 1), (1, 3), (2, 3), (3, 0), (3, 1)]
9.30915379524231



Heuristica 1
 [(1, 1), (1, 2), (1, 0), (1, 3), (3, 2), (2, 1), (3, 0), (3, 1), (2, 2), (3, 3)]
0.3299121856689453



Heuristica 2
 [(1, 0), (0, 2), (0, 3), (2, 0), (3, 2), (3, 3)]
25.923059463500977

Heuristica 3
 [(0, 0), (0, 1), (3, 0), (2, 3), (1, 3), (3, 1)]
0.06909370422363281


1536271410.9180093

In [30]:
five_matrix = [[0,1,0,0,0],[1,0,0,0,0],[0,0,1,1,0],[0,1,1,1,1],[0,0,1,1,0]]

t = Tick()

t.tick()
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix)
n = breadth_first_search(problem)
print("\nBlind\n",n.solution())
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix,h1)
n = astar_search(problem,problem.h)
print("\nHeuristica 1\n",n.solution())
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix,h2)
n = astar_search(problem,problem.h)
print("\nHeuristica 2\n",n.solution())
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix,h3)
n = astar_search(problem,problem.h)
print("\nHeuristica 3\n",n.solution())
print(t.tock())
t.tick()

3.314018249511719e-05

Blind
 [(0, 0), (3, 2), (3, 3)]
0.16147112846374512

Heuristica 1
 [(3, 2), (3, 3), (0, 0)]
0.0038847923278808594



Heuristica 2
 [(0, 0), (3, 2), (3, 3)]
0.0689687728881836



Heuristica 3
 [(3, 2), (0, 0), (3, 3)]
9.055302619934082


1536272043.9873765

In [14]:
four_matrix = [
[0,0,1,0],
[0,1,0,1],
[0,0,1,0],
[0,0,0,0]
]
ones_matrix = [
[1,1,1,1],
[1,1,1,1],
[1,1,1,1],
[1,1,1,1]
]
five_matrix = [
[0,1,0,0,0],
[1,0,0,0,0],
[0,0,1,1,0],
[0,1,1,1,1],
[0,0,1,1,0]
]
six = [
[0,1,0,0,0,1],
[1,0,0,0,1,0],
[0,0,1,1,0,1],
[0,1,1,1,1,0],
[0,0,1,1,0,0],
[0,0,0,0,0,0]
]

def h3(node):
    if(node.action is None):
        return 100
    else:
        return num_neighbors(node.state, node.action[0], node.action[1], 1)
t = Tick()

t.tick()
print(t.tock())
t.tick()
print("------------ Four Matrix -------------")
problem = CleanPuzzle(four_matrix)
n = breadth_first_search(problem)
print_result("Blind",n)
print(t.tock())
t.tick()
problem = CleanPuzzle(four_matrix,h1)
n = astar_search(problem,problem.h)
print_result("Heuristica 1",n)
print(t.tock())
t.tick()
problem = CleanPuzzle(four_matrix,h2)
n = astar_search(problem,problem.h)
print_result("Heuristica 2",n)
print(t.tock())
t.tick()

print("------------ Ones Matrix -------------")
problem = CleanPuzzle(ones_matrix)
n = breadth_first_search(problem)
print_result("Blind",n)
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix,h1)
n = astar_search(problem,problem.h)
print_result("Heuristica 1",n)
print(t.tock())
t.tick()
problem = CleanPuzzle(ones_matrix,h2)
n = astar_search(problem,problem.h)
print_result("Heuristica 2",n)
print(t.tock())
t.tick()

print("------------ five_matrix Matrix -------------")
problem = CleanPuzzle(five_matrix)
n = breadth_first_search(problem)
print_result("Blind",n)
print(t.tock())
t.tick()
problem = CleanPuzzle(five_matrix,h1)
n = astar_search(problem,problem.h)
print_result("Heuristica 1",n)
print(t.tock())
t.tick()
problem = CleanPuzzle(five_matrix,h2)
n = astar_search(problem,problem.h)
print_result("Heuristica 2",n)
print(t.tock())
t.tick()

print("------------ six Matrix -------------")
problem = CleanPuzzle(six)
n = breadth_first_search(problem)
print_result("Blind",n)
print(t.tock())
t.tick()
problem = CleanPuzzle(six,h1)
n = astar_search(problem,problem.h)
print_result("Heuristica 1",n)
print(t.tock())
t.tick()
problem = CleanPuzzle(six,h2)
n = astar_search(problem,problem.h)
print_result("Heuristica 2",n)
print(t.tock())
t.tick()

3.6716461181640625e-05
------------ Four Matrix -------------
----------
 Blind :


Accion:  None 

  0 1 2 3  
0 0 0 0 0 
1 0 1 0 0 
2 1 0 1 0 
3 0 1 0 0 

Accion:  (1, 2) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 0 
3 0 0 0 0 
0.0042912960052490234
----------
 Heuristica 1 :


Accion:  None 

  0 1 2 3  
0 0 0 0 0 
1 0 1 0 0 
2 1 0 1 0 
3 0 1 0 0 

Accion:  (1, 2) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 0 
3 0 0 0 0 
0.01236724853515625
----------
 Heuristica 2 :


Accion:  None 

  0 1 2 3  
0 0 0 0 0 
1 0 1 0 0 
2 1 0 1 0 
3 0 1 0 0 

Accion:  (1, 2) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 0 
3 0 0 0 0 
0.03486347198486328
------------ Ones Matrix -------------


----------
 Blind :


Accion:  None 

  0 1 2 3  
0 1 1 1 1 
1 1 1 1 1 
2 1 1 1 1 
3 1 1 1 1 

Accion:  (0, 0) 

  0 1 2 3  
0 1 0 1 1 
1 0 1 1 1 
2 1 1 1 1 
3 1 1 1 1 

Accion:  (0, 1) 

  0 1 2 3  
0 0 0 1 1 
1 0 0 1 1 
2 0 1 1 1 
3 1 1 1 1 

Accion:  (1, 3) 

  0 1 2 3  
0 0 0 1 1 
1 0 0 1 1 
2 0 0 1 1 
3 0 1 0 1 

Accion:  (2, 3) 

  0 1 2 3  
0 0 0 1 1 
1 0 0 1 1 
2 0 0 0 1 
3 0 0 0 0 

Accion:  (3, 0) 

  0 1 2 3  
0 0 0 0 1 
1 0 0 1 0 
2 0 0 0 1 
3 0 0 0 0 

Accion:  (3, 1) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 0 
3 0 0 0 0 
9.640665531158447


----------
 Heuristica 1 :


Accion:  None 

  0 1 2 3  
0 1 1 1 1 
1 1 1 1 1 
2 1 1 1 1 
3 1 1 1 1 

Accion:  (1, 1) 

  0 1 2 3  
0 1 0 1 1 
1 0 1 0 1 
2 1 0 1 1 
3 1 1 1 1 

Accion:  (1, 2) 

  0 1 2 3  
0 1 0 1 1 
1 0 0 0 1 
2 0 0 0 1 
3 1 0 1 1 

Accion:  (1, 0) 

  0 1 2 3  
0 0 0 0 1 
1 0 1 0 1 
2 0 0 0 1 
3 1 0 1 1 

Accion:  (1, 3) 

  0 1 2 3  
0 0 0 0 1 
1 0 1 0 1 
2 0 1 0 1 
3 0 0 0 1 

Accion:  (3, 2) 

  0 1 2 3  
0 0 0 0 1 
1 0 1 0 0 
2 0 1 1 1 
3 0 0 0 0 

Accion:  (2, 1) 

  0 1 2 3  
0 0 0 1 1 
1 0 0 0 1 
2 0 1 0 1 
3 0 0 0 0 

Accion:  (3, 0) 

  0 1 2 3  
0 0 0 0 1 
1 0 0 0 0 
2 0 1 0 1 
3 0 0 0 0 

Accion:  (3, 1) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 1 0 
2 0 1 0 0 
3 0 0 0 0 

Accion:  (2, 2) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 1 
3 0 0 1 0 

Accion:  (3, 3) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 0 
3 0 0 0 0 
0.3718733787536621


----------
 Heuristica 2 :


Accion:  None 

  0 1 2 3  
0 1 1 1 1 
1 1 1 1 1 
2 1 1 1 1 
3 1 1 1 1 

Accion:  (1, 0) 

  0 1 2 3  
0 0 1 0 1 
1 1 0 1 1 
2 1 1 1 1 
3 1 1 1 1 

Accion:  (0, 2) 

  0 1 2 3  
0 0 1 0 1 
1 0 0 1 1 
2 1 0 1 1 
3 0 1 1 1 

Accion:  (0, 3) 

  0 1 2 3  
0 0 1 0 1 
1 0 0 1 1 
2 0 0 1 1 
3 0 0 1 1 

Accion:  (2, 0) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 1 
2 0 0 1 1 
3 0 0 1 1 

Accion:  (3, 2) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 1 
3 0 0 1 0 

Accion:  (3, 3) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 0 
3 0 0 0 0 
28.86165738105774
------------ five_matrix Matrix -------------
----------
 Blind :


Accion:  None 

  0 1 2 3 4  
0 0 1 0 0 0 
1 1 0 0 1 0 
2 0 0 1 1 1 
3 0 0 1 1 1 
4 0 0 0 1 0 

Accion:  (0, 0) 

  0 1 2 3 4  
0 0 0 0 0 0 
1 0 0 0 1 0 
2 0 0 1 1 1 
3 0 0 1 1 1 
4 0 0 0 1 0 

Accion:  (3, 2) 

  0 1 2 3 4  
0 0 0 0 0 0 
1 0 0 0 0 0 
2 0 0 0 1 0 
3 0 0 1 0 1 
4 0 0 0 1 0 

Accion:  (3, 3) 

  0 1 2 3 4  
0 0 0 0 0 0 
1 0 0 0 0 0 
2 0 0 0 0 0 


----------
 Heuristica 2 :


Accion:  None 

  0 1 2 3 4  
0 0 1 0 0 0 
1 1 0 0 1 0 
2 0 0 1 1 1 
3 0 0 1 1 1 
4 0 0 0 1 0 

Accion:  (0, 0) 

  0 1 2 3 4  
0 0 0 0 0 0 
1 0 0 0 1 0 
2 0 0 1 1 1 
3 0 0 1 1 1 
4 0 0 0 1 0 

Accion:  (3, 2) 

  0 1 2 3 4  
0 0 0 0 0 0 
1 0 0 0 0 0 
2 0 0 0 1 0 
3 0 0 1 0 1 
4 0 0 0 1 0 

Accion:  (3, 3) 

  0 1 2 3 4  
0 0 0 0 0 0 
1 0 0 0 0 0 
2 0 0 0 0 0 
3 0 0 0 0 0 
4 0 0 0 0 0 
0.15395498275756836
------------ six Matrix -------------


----------
 Blind :


Accion:  None 

  0 1 2 3 4 5  
0 0 1 0 0 0 0 
1 1 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 1 1 1 0 
4 0 1 0 1 0 0 
5 1 0 1 0 0 0 

Accion:  (0, 0) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 1 1 1 0 
4 0 1 0 1 0 0 
5 1 0 1 0 0 0 

Accion:  (1, 5) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 1 1 1 0 
4 0 0 0 1 0 0 
5 0 0 0 0 0 0 

Accion:  (3, 2) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 0 0 0 
2 0 0 0 1 0 0 
3 0 0 1 0 1 0 
4 0 0 0 1 0 0 
5 0 0 0 0 0 0 

Accion:  (3, 3) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 0 0 0 
2 0 0 0 0 0 0 
3 0 0 0 0 0 0 
4 0 0 0 0 0 0 
5 0 0 0 0 0 0 
123.66450047492981
----------
 Heuristica 1 :


Accion:  None 

  0 1 2 3 4 5  
0 0 1 0 0 0 0 
1 1 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 1 1 1 0 
4 0 1 0 1 0 0 
5 1 0 1 0 0 0 

Accion:  (2, 4) 

  0 1 2 3 4 5  
0 0 1 0 0 0 0 
1 1 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 0 1 1 0 
4 0 0 0 0 0 0 
5 1 0 0 0 0 0 

Accion:  (3, 2) 

  0 1 2 3 4 5  
0 0 1 0 0 0 0 
1 1 0 0 0 0

----------
 Heuristica 2 :


Accion:  None 

  0 1 2 3 4 5  
0 0 1 0 0 0 0 
1 1 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 1 1 1 0 
4 0 1 0 1 0 0 
5 1 0 1 0 0 0 

Accion:  (0, 0) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 1 1 1 0 
4 0 1 0 1 0 0 
5 1 0 1 0 0 0 

Accion:  (1, 5) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 1 1 1 0 
4 0 0 0 1 0 0 
5 0 0 0 0 0 0 

Accion:  (3, 2) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 0 0 0 
2 0 0 0 1 0 0 
3 0 0 1 0 1 0 
4 0 0 0 1 0 0 
5 0 0 0 0 0 0 

Accion:  (3, 3) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 0 0 0 
2 0 0 0 0 0 0 
3 0 0 0 0 0 0 
4 0 0 0 0 0 0 
5 0 0 0 0 0 0 
7.888329029083252


1536358323.7131953

In [15]:
def h3(node):
    if(node.action is None):
        return 100
    else:
        return num_neighbors(node.state, node.action[0], node.action[1], 2)
t = Tick()

t.tick()
problem = CleanPuzzle(four_matrix,h3)
n = astar_search(problem,problem.h)
print_result("Heuristica 3",n)
print(t.tock())
t.tick()

t.tick()
problem = CleanPuzzle(ones_matrix,h3)
n = astar_search(problem,problem.h)
print_result("Heuristica 3",n)
print(t.tock())
t.tick()

t.tick()
problem = CleanPuzzle(five_matrix,h3)
n = astar_search(problem,problem.h)
print_result("Heuristica 3",n)
print(t.tock())
t.tick()

t.tick()
problem = CleanPuzzle(six,h3)
n = astar_search(problem,problem.h)
print_result("Heuristica 3",n)
print(t.tock())
t.tick()

----------
 Heuristica 3 :


Accion:  None 

  0 1 2 3  
0 0 0 0 0 
1 0 1 0 0 
2 1 0 1 0 
3 0 1 0 0 

Accion:  (1, 2) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 0 
3 0 0 0 0 
0.011883258819580078
----------
 Heuristica 3 :


Accion:  None 

  0 1 2 3  
0 1 1 1 1 
1 1 1 1 1 
2 1 1 1 1 
3 1 1 1 1 

Accion:  (0, 0) 

  0 1 2 3  
0 1 0 1 1 
1 0 1 1 1 
2 1 1 1 1 
3 1 1 1 1 

Accion:  (0, 1) 

  0 1 2 3  
0 0 0 1 1 
1 0 0 1 1 
2 0 1 1 1 
3 1 1 1 1 

Accion:  (3, 0) 

  0 1 2 3  
0 0 0 0 1 
1 0 0 1 0 
2 0 1 1 1 
3 1 1 1 1 

Accion:  (2, 3) 

  0 1 2 3  
0 0 0 0 1 
1 0 0 1 0 
2 0 1 0 1 
3 1 0 1 0 

Accion:  (1, 3) 

  0 1 2 3  
0 0 0 0 1 
1 0 0 1 0 
2 0 0 0 1 
3 0 0 0 0 

Accion:  (3, 1) 

  0 1 2 3  
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 0 
3 0 0 0 0 
0.1180729866027832
----------
 Heuristica 3 :


Accion:  None 

  0 1 2 3 4  
0 0 1 0 0 0 
1 1 0 0 1 0 
2 0 0 1 1 1 
3 0 0 1 1 1 
4 0 0 0 1 0 

Accion:  (3, 3) 

  0 1 2 3 4  
0 0 1 0 0 0 
1 1 0 0 1 0 
2 0 0 1 0 1 
3 0 0 0 1 0 
4 0 0 0 0 0 

Accion:  (

----------
 Heuristica 3 :


Accion:  None 

  0 1 2 3 4 5  
0 0 1 0 0 0 0 
1 1 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 1 1 1 0 
4 0 1 0 1 0 0 
5 1 0 1 0 0 0 

Accion:  (1, 5) 

  0 1 2 3 4 5  
0 0 1 0 0 0 0 
1 1 0 0 1 0 0 
2 0 0 1 1 1 0 
3 0 0 1 1 1 0 
4 0 0 0 1 0 0 
5 0 0 0 0 0 0 

Accion:  (3, 3) 

  0 1 2 3 4 5  
0 0 1 0 0 0 0 
1 1 0 0 1 0 0 
2 0 0 1 0 1 0 
3 0 0 0 1 0 0 
4 0 0 0 0 0 0 
5 0 0 0 0 0 0 

Accion:  (0, 0) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 1 0 0 
2 0 0 1 0 1 0 
3 0 0 0 1 0 0 
4 0 0 0 0 0 0 
5 0 0 0 0 0 0 

Accion:  (3, 2) 

  0 1 2 3 4 5  
0 0 0 0 0 0 0 
1 0 0 0 0 0 0 
2 0 0 0 0 0 0 
3 0 0 0 0 0 0 
4 0 0 0 0 0 0 
5 0 0 0 0 0 0 
1.5680429935455322


1536358341.0134234